# Load and test trained model

## Load libraries

In [1]:
import numpy as np
import torch

import torch.nn as nn
from enduro_lstm import *

In [2]:
def conf_cuda(use_cuda):
    
    if use_cuda:
        
        # torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
        is_cuda = torch.cuda.is_available()

        # If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
        if is_cuda:
            device = torch.device("cuda")
            print("GPU is available")
        else:
            device = torch.device("cpu")
            print("GPU not available, CPU used")
    else:
        device = torch.device("cpu")
        print("Selected CPU")
    return device

In [3]:
device = conf_cuda(False)

Selected CPU


In [4]:
device

device(type='cpu')

In [5]:
device.type

'cpu'

## Set configurations

In [6]:
import os
dir_path = "models/RNN_chunked_m1-2-3-4-5-6-7-8_f1to1020_epoch5000_H200" + "/"
arr = os.listdir(f'./{dir_path}')
for i in range(len(arr)):
    print(arr[i])

loss_file.txt
RNN_chunked_m1-2-3-4-5-6-7-8_f1to1020_epoch5000_H200
train_loss.png
train_loss_arr.npz


In [7]:
model_path = dir_path + "RNN_chunked_m1-2-3-4-5-6-7-8_f1to1020_epoch5000_H200"

In [8]:
start_match = 9
end_match = 9

hidden_neurons = 200
zigzag = False
is_softmax = True

start_frame = 1
end_frame = 1030

In [9]:
data_path = r"../1-generate/data/"

use_cuda = False

In [10]:
if zigzag:
    output_size = 2
else:
    output_size = 9

## Load trained model

In [11]:
%load_ext autoreload
%autoreload 2

In [12]:
from setup_model import *
from setup_model_types import *

In [13]:
def load_checkpoint(model, filename='checkpoint.pth.tar'):
    
    print("=> loading checkpoint '{}'".format(filename))
    checkpoint = torch.load(filename, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])

    return model, checkpoint['optimizer']['state'][0]['step'], checkpoint['losslogger']

In [14]:
model = RNNModel(device=device, input_size=10000, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1)
model, last_epoch, last_logger = load_checkpoint(model, model_path)

=> loading checkpoint 'models/RNN_chunked_m1-2-3-4-5-6-7-8_f1to1020_epoch5000_H200/RNN_chunked_m1-2-3-4-5-6-7-8_f1to1020_epoch5000_H200'


In [15]:
ACTIONS_LIST = get_actions_list(zigzag=zigzag)

In [16]:
num_of_frames_arr = []
frames_arr = []
actions_arr = []

start_frame = 1
end_frame = 1000

for m in range(start_match, end_match + 1):
    
    num_of_frames, frames, actions, rewards, lifes = load_npz(data_path, m)
    frames = frames[start_frame - 1:end_frame, 30:130, 10:110]
    actions = actions[start_frame - 1:end_frame]
    
    action_one_hot = [prepare_action_data(i, ACTIONS_LIST) for i in actions]
    actions = np.array(action_one_hot)
    actions = actions.reshape(len(actions), -1)
    
    frames_arr.append(frames)
    actions_arr.append(actions)
    num_of_frames_arr.append(end_frame - start_frame + 1) 

X_train = np.array(frames_arr)/255
Y_train = np.array(actions_arr)
num_of_frames_arr = np.array(num_of_frames_arr)

X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

Successfully loaded NPZ.


In [17]:
X_train = X_train.reshape(1, end_frame-start_frame+1, -1)

In [18]:
X_train.shape, Y_train.shape

(torch.Size([1, 1000, 10000]), torch.Size([1, 1000, 9]))

## Prepare cell with trained model 

In [19]:
rnncell = nn.RNNCell(10000, hidden_neurons)
linear = nn.Linear(hidden_neurons, output_size)
output = nn.Softmax()

In [20]:
output

Softmax(dim=None)

In [21]:
rnncell.weight_ih = model.rnn.weight_ih_l0
rnncell.weight_hh = model.rnn.weight_hh_l0
rnncell.bias_hh = model.rnn.bias_hh_l0
rnncell.bias_ih = model.rnn.bias_ih_l0
linear.weight = model.fc.weight
linear.bias = model.fc.bias

In [22]:
hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

In [23]:
print(model.rnn.weight_ih_l0.shape)
print(model.rnn.weight_hh_l0.shape)
print(model.rnn.bias_ih_l0.shape)
print(model.rnn.bias_hh_l0.shape)
print(model.fc.weight.shape)
print(model.fc.bias.shape)

torch.Size([200, 10000])
torch.Size([200, 200])
torch.Size([200])
torch.Size([200])
torch.Size([9, 200])
torch.Size([9])


## Testing outputs of model

In [24]:
X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

<ipython-input-24-19911837e977>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train).float()
<ipython-input-24-19911837e977>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_train = torch.tensor(Y_train).float()


In [25]:
X_train.shape

torch.Size([1, 1000, 10000])

In [26]:
hx = torch.zeros(1, hidden_neurons)
out_arr = []
for i in range(1000):
    step_input = X_train[0][i]
    step_input = step_input.reshape(1, -1)
    hx = rnncell(step_input, hx)
    out = linear(hx)
    out = output(out)
    out_arr.append(out)

<ipython-input-26-5d4a2a5081cf>:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = output(out)


In [27]:
Y_train = Y_train.reshape(-1, len(ACTIONS_LIST))

In [28]:
acertou = 0
errou = 0
for i in range(1000):
    if torch.argmax(Y_train[i]) == torch.argmax(out_arr[i]):
        acertou += 1
    else:
        errou += 1
        
print(acertou)
print(errou)

630
370


In [29]:
acertou/1000

0.63

## Play Gym Enduro

In [30]:
import gym
import time
from PIL import Image

In [31]:
if zigzag:
        
    ACTIONS = {
        "right": 2,
        "left": 3,
    }

else:

    ACTIONS = {
        "noop": 0,
        "accelerate": 1,
        "right": 2,
        "left": 3,
        "break": 4,
        "right_break": 5,
        "left_break": 6,
        "right_accelerate": 7,
        "left_accelerate": 8,
    }

In [32]:
y_min, y_max, x_min, x_max = 30, 130, 10, 110
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [33]:
sleep_time = 0.05

In [34]:
env = gym.make("Enduro-v0")
frame = env.reset()
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

env.render()
time.sleep(1)

for _ in range(1000):
    
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(1, -1)
    frame = torch.tensor(frame)/255
    
    hx = rnncell(frame, hx)
    out = linear(hx)
    action = output(out)
    
    action = list(ACTIONS.values())[torch.argmax(action, axis=1)]
    print(action)
    frame, reward, done, info = env.step(action)

<ipython-input-34-63d7e5fce817>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  action = output(out)


3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
3
0
0
0
0
0
0
0
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
